# Detection with VGG2
In this notebook, we are using VGG16 for classification. This time image augmentation is performed beforehand on the dataset, so no need for weighed random sampling.

In [1]:
!pip install git+https://github.com/obsessor-ak1/Skin_Cancer_Detection_HAM10000.git --no-deps --force-reinstall --no-cache-dir

  Cloning https://github.com/obsessor-ak1/Skin_Cancer_Detection_HAM10000.git to /tmp/pip-req-build-talr7ey0
  Running command git clone --filter=blob:none --quiet https://github.com/obsessor-ak1/Skin_Cancer_Detection_HAM10000.git /tmp/pip-req-build-talr7ey0
  Resolved https://github.com/obsessor-ak1/Skin_Cancer_Detection_HAM10000.git to commit c6658e1b657c9cc7eac19ba0a2b2ec41dae73cb7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for skin_cancer_detection: filename=skin_cancer_detection-0.1.0-py3-none-any.whl size=15719 sha256=f91d657775abadedc4a5cda986f9a6a5935d722fcf9274bd20e0a67b42fc21f4
  Stored in directory: /tmp/pip-ephem-wheel-cache-3yijj3pq/wheels/1c/73/8d/15507a4fb68edbf3c8bf56d26a749022d0075ab6b50ec069ff
Successfully built skin_cancer_detection


In [2]:
from functools import partial
import os

import kagglehub
from sklearn.metrics import accuracy_score, precision_score, recall_score
import torch
from torch.utils.data import DataLoader
from torchvision.io import read_image
from torchvision.transforms import v2 as tfs
from torchvision.datasets import ImageFolder

from exp_tools.basic_utils import init_module
from exp_tools.metrics import ham10000_precision, ham10000_recall
from exp_tools.training.trainer import Trainer
from exp_tools.modules import vgg_layers

## Loading the Dataset
First, let's load the dataset from the image folders.

In [3]:
BATCH_SIZE = 64
normalization = tfs.Normalize(
    mean=[0.5, 0.5, 0.5],
    std=[0.5, 0.5, 0.5]
)
train_transform = tfs.Compose([
    tfs.ToImage(),
    tfs.ToDtype(torch.float32, scale=1.0),
    normalization
])
test_transform = tfs.Compose([
    tfs.ToImage(),
    tfs.Resize((224, 224)),
    tfs.ToDtype(torch.float32, scale=1.0),
    normalization
])

In [4]:
path = kagglehub.dataset_download("ak11chp/ham10000-augmented8000-1")
trainset = ImageFolder(
    os.path.join(path, "HMNIST_AUG", "train"),
    transform=train_transform,
    target_transform=torch.tensor,
    loader=read_image
)
testset = ImageFolder(
    os.path.join(path, "HMNIST_AUG", "test"),
    transform=test_transform,
    target_transform=torch.tensor,
    loader=read_image
)

In [5]:
train_loader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
test_loader = DataLoader(testset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

## Training the Model
Now, we are going to load our vgg16 model and train it.

In [6]:
class VGG16_BN(torch.nn.Module):
    """A customized vgg 16 model."""
    def __init__(self, num_classes=7, dropout=0.5):
        super().__init__()
        self.features = torch.nn.Sequential(
            vgg_layers.make_blocks(3, 64, 2, end_1x1=True, with_bn=True),
            vgg_layers.make_blocks(64, 128, 2, end_1x1=True, with_bn=True),
            vgg_layers.make_blocks(128, 256, 3, end_1x1=True, with_bn=True),
            vgg_layers.make_blocks(256, 512, 3, end_1x1=True, with_bn=True),
            vgg_layers.make_blocks(512, 512, 3, end_1x1=True, with_bn=True)
        )
        self.pool = torch.nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = vgg_layers.get_classifier(
            num_classes=num_classes,
            dropout=dropout
        )

    def forward(self, X):
        X = self.features(X)
        X = self.pool(X)
        X = torch.flatten(X, start_dim=1)
        X = self.classifier(X)
        return X

In [7]:
model = VGG16_BN(num_classes=7)
model

VGG16_BN(
  (features): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (1): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1))
      (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=

In [8]:
metrics = {
    "accuracy": accuracy_score,
    "average_precision": partial(precision_score, average="macro", zero_division=0.0),
    "average_recall": partial(recall_score, average="macro", zero_division=0.0),
    "precision": ham10000_precision,
    "recall": ham10000_recall
}

In [9]:
trainset.class_to_idx

{'akiec': 0, 'bcc': 1, 'bkl': 2, 'df': 3, 'mel': 4, 'nv': 5, 'vasc': 6}

In [10]:
weights = torch.tensor([5.0 if cls == "mel" else 1.0 for cls in trainset.classes], device="cuda") # higher weight for melanoma class
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, eps=0.1)
criterion = torch.nn.CrossEntropyLoss(weight=weights)

In [12]:
trainer = Trainer(
    max_epochs=10, device="cuda", metrics=metrics
)

In [ ]:
%%time
trainer.fit(model, criterion, optimizer, train_loader, test_loader)

Epoch 1/10
[--------------------------------------------------] - batch: 875/875 - 100.00 complete
Train loss: 1.2730767427853176
Val loss: 2.414516713514746
Epoch 2/10
[--------------------------------------------------] - batch: 875/875 - 100.00 complete
Train loss: 0.9372495118209294
Val loss: 2.6466434244140684
Epoch 3/10
[--------------------------------------------------] - batch: 875/875 - 100.00 complete
Train loss: 0.8097257800442832
Val loss: 2.5032690970862848
Epoch 4/10
[--------------------------------------------------] - batch: 875/875 - 100.00 complete
Train loss: 0.6956926997389112
Val loss: 2.943629527946867
Epoch 5/10
[--------------------------------------------------] - batch: 875/875 - 100.00 complete
Train loss: 0.6260288433006832
Val loss: 2.579224190072393
Epoch 6/10
[--------------------------------------------------] - batch: 875/875 - 100.00 complete
Train loss: 0.5582365260635104
Val loss: 2.5285801444237292
Epoch 7/10
[-------------------------------------

In [ ]:
trainer.current_history.plot_history()

In [18]:
torch.cuda.empty_cache()